# Torchserve Setup for the First Model

### 1. Download Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

### 2. Save Locally

In [ ]:
from pathlib import Path

In [ ]:
import shutil

In [ ]:
cur_dir = Path.cwd()
base_dir = cur_dir/'model-1'
model_dir = base_dir/'model'
code_dir = base_dir/'upload_code'

In [ ]:
shutil.rmtree(str(model_dir))

In [ ]:
tokenizer.save_pretrained(model_dir)

In [ ]:
model.save_pretrained(model_dir)

### 3. Copy inference code inside the model directory
```
model/
|
|- config.json
|- pytorch_model.bin
|- special_tokens_map.json
|- tokenizer_config.json
|- vocab.txt
|- code/
   |- inference.py
```

In [ ]:
shutil.copytree(code_dir, str(model_dir/'code'))

### 4. Install Dependencies
This is to keep the model decoupled from others

In [ ]:
import subprocess
import os

Important! The python version used to install the dependencies must match the deployed image

In [ ]:
my_env = os.environ.copy()
my_env['PYENV_VERSION'] = '3.6.10'

subprocess.check_call(["pyenv", "exec", "python", "-m", "pip", "install", "--target", str(model_dir/'code'), "transformers==4.2.2"], env=my_env)

### 5. Add Manifest File

In [ ]:
subprocess.check_call([
    "torch-model-archiver", "--model-name", "model", 
    "--handler", "torchserve/handler_service.py",
    "--export-path", "model-1",
    "--version", "1",
    "--archive-format", "no-archive",
    "-f"
])

### 6. Package model into a tar

In [ ]:
import tarfile

In [ ]:
with tarfile.open(base_dir/'model.tar.gz', 'w:gz') as tar:
    tar.add(model_dir, arcname='.')

### 7. Upload model package to S3

In [ ]:
import boto3
s3 = boto3.resource("s3")

In [ ]:
s3_bucket = "mlops-mme-torchserve-deployment-bucket-validation"

In [ ]:
bucket = s3.Bucket(s3_bucket)
bucket.upload_file(str(base_dir/'model.tar.gz'), 'model-1b.tar.gz')